In [ ]:
!nvidia-smi

Tue Jun 18 16:22:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install libraries

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git
!pip install flax jsonlines ftfy wandb
!pip install -U tqdm
!pip install git+https://github.com/deepmind/optax.git

# Aggiunto install
# Si dovrebbe restartare il runtime dopo questo install

! pip install -U accelerate
! pip install -U transformers
! pip install codecarbon

In [ ]:
!pip install -U "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.6/81.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.1.1.17 which is incompatible.


In [ ]:
%%capture
!git clone https://github.com/arampacha/CLIP-rsicd

In [ ]:
import sys
sys.path += ['CLIP-rsicd']

## Imports

In [ ]:
import jax.tools.colab_tpu

try:
    # jax.tools.colab_tpu.setup_tpu()
    half_precision_dtype = "bfloat16"
except KeyError as e:
    half_precision_dtype = "float16"
    print("No TPU detected")


device_count = jax.device_count()
print(device_count)
jax.devices()

1


[cuda(id=0)]

In [ ]:
import logging
import math
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional, List
import json
import jsonlines
import shutil
import numpy as np
import torch

from flax import training
from tqdm.auto import tqdm

import jax
import jax.numpy as jnp
import optax
import transformers
from flax import jax_utils, traverse_util
from flax.jax_utils import unreplicate
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from flax.training.checkpoints import save_checkpoint, restore_checkpoint
from flax.serialization import to_bytes, from_bytes
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    FlaxCLIPModel,
    CLIPProcessor,
    CLIPTokenizer,
    TrainingArguments,
    is_tensorboard_available,
    IntervalStrategy

)
from transformers.testing_utils import CaptureLogger

from importlib.util import find_spec

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
from utils.data import ImageTextDataset, Transform

## Data

In this notebook we will be using [RSICD dataset](). A copy of this dataset is shared by the authors on google drive. To use it one can simply add a shrtcut to own gdrive. In my case it's `RSICD` folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
path = Path(root_dir + 'RSICD/')
#path = Path(root_dir + 'WHU/')

In [ ]:
%%capture
# Fix delle cartelle
# !rm -rf /content/data/*

!unzip /content/gdrive/MyDrive/rscid-dataset-remotesensing.zip -d /content/data/

!rm -rf /content/data/rsicd_images/*


!mv /content/data/dataset/RSICD/RSICD_images/ /content/data/RSICD_images/


In [ ]:
files = sorted(os.listdir('/content/data/RSICD_images'))  # Sort to ensure consistent order
port_found = False

for file in files:
    if port_found:
        file_path = os.path.join('/content/data/RSICD_images', file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
    elif file.startswith("port"):
        port_found = True
        print(f"Found file starting with 'port': {file}")

Found file starting with 'port': port_1.jpg
Deleted file: /content/data/RSICD_images/port_10.jpg
Deleted file: /content/data/RSICD_images/port_100.jpg
Deleted file: /content/data/RSICD_images/port_101.jpg
Deleted file: /content/data/RSICD_images/port_102.jpg
Deleted file: /content/data/RSICD_images/port_103.jpg
Deleted file: /content/data/RSICD_images/port_104.jpg
Deleted file: /content/data/RSICD_images/port_105.jpg
Deleted file: /content/data/RSICD_images/port_106.jpg
Deleted file: /content/data/RSICD_images/port_107.jpg
Deleted file: /content/data/RSICD_images/port_108.jpg
Deleted file: /content/data/RSICD_images/port_109.jpg
Deleted file: /content/data/RSICD_images/port_11.jpg
Deleted file: /content/data/RSICD_images/port_110.jpg
Deleted file: /content/data/RSICD_images/port_111.jpg
Deleted file: /content/data/RSICD_images/port_112.jpg
Deleted file: /content/data/RSICD_images/port_113.jpg
Deleted file: /content/data/RSICD_images/port_114.jpg
Deleted file: /content/data/RSICD_images

In [ ]:
!mv /content/data/RSICD_images/ /content/data/rsicd_images
#!mkdir /content/data/
#!cp -r /content/gdrive/MyDrive/WHU /content/data/rsicd_images

In [ ]:
# !cp /content/gdrive/MyDrive/textaug_train_nostro.jsonl /content/data
# !cp /content/gdrive/MyDrive/textaug_valid_nostro.jsonl /content/data

cp: cannot stat '/content/gdrive/MyDrive/textaug_train_nostro.jsonl': No such file or directory
cp: cannot stat '/content/gdrive/MyDrive/textaug_valid_nostro.jsonl': No such file or directory


We'll be using augmented captions for the images. The files containing those are available in our [github repo]().

In [ ]:
%%capture
!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_train_rsicd.jsonl
!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_valid_rsicd.jsonl
#!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_test_rsicd.jsonl

Each line in the `jsonl` file has two fields: `filename` and `captions`:

In [ ]:
with jsonlines.open("/content/data/textaug_valid_rsicd.jsonl", "r") as reader:
#with jsonlines.open("/content/data/textaug_train_nostro.jsonl", "r") as reader:
    line = next(iter(reader))
print(f"filename: {line['filename']}\n")
print("captions:")
for i, caption in enumerate(line["captions"]):
    print(f"{i+1}) {caption}")

filename: rsicd_images/airport_61.jpg

captions:
1) the asphalted and airport runways divide the field into several rounded rectangles arranged next to which buildings and a road are located.
2) the tarmac and airport runways divide the field into several orderly arranged rounded rectangles  next to which is buildings and a road .
3) a brown ground divided by the grey runway .
4) we can see a simple termial building and an apron connceted with runways .
5) some building with a parking lot are near an airport with several runways .


## Training arguments

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker(
    project_name="RS-ZSL-UNIVPM",
    measure_power_secs=15,
    output_dir="./emissions",
    log_level="debug",
    save_to_file=True
)

tracker.start()

[codecarbon WARNING @ 17:48:59] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:48:59] [setup] RAM Tracking...
[codecarbon INFO @ 17:48:59] [setup] GPU Tracking...
[codecarbon INFO @ 17:48:59] Tracking Nvidia GPU via pynvml
[codecarbon DEBUG @ 17:48:59] GPU available. Starting setup
[codecarbon INFO @ 17:48:59] [setup] CPU Tracking...
[codecarbon DEBUG @ 17:48:59] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 17:48:59] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 17:48:59] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 17:48:59] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:49:00] We saw t

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the model weights should be initialized and trained. Choose one of `[float32, float16, bfloat16]`."
        },
    )
    save_optimizer: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to store full train state including optimizer."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    data_dir: Optional[str] = field(
        default=None, metadata={"help": "Path to local folder containing data files."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file (a jsonlines file)."},
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    text_column_name: Optional[str] = field(
            default='text',
            metadata={"help": "Column containing main text data."},
    )
    augment_images: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    augment_captions: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    captions_per_image: Optional[int] = field(
        default=5,
        metadata={"help": "Number of captions per image to use when creating train dataset."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`validation_file` should be a csv, a json or a txt file."


@dataclass
class ImageAugmentationArguments:
    """
    Arguments for image augmentations configuration
    """
    random_horizontal_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random horizontal flip" }
    )
    random_vertical_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random vartical flip" }
    )


In [ ]:
training_args = TrainingArguments(
    output_dir = "./tmp/model1",
    overwrite_output_dir = False,
    do_train = True,
    do_eval = True,
    evaluation_strategy = "steps",
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    gradient_accumulation_steps = 1,
    learning_rate = 5e-07,
    weight_decay = 0.1,
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    adam_epsilon = 5e-06,
    max_grad_norm = 1.0,
    num_train_epochs = 5,
    max_steps = -1,
    lr_scheduler_type = "linear",
    warmup_ratio = 0.2,
    # warmup_steps = 0,
    logging_first_step = False,
    logging_steps = 50,
    save_strategy = "epoch",
    seed = 42,
    dataloader_drop_last = True,
    eval_steps = 200,
    run_name = None,
    adafactor = False,
    report_to = "all",
    skip_memory_metrics = True,
    resume_from_checkpoint = None,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_args = ModelArguments(
    model_name_or_path = "openai/clip-vit-base-patch32",
    dtype = half_precision_dtype,
    save_optimizer = False,
)

In [ ]:
data_args = DataTrainingArguments(
    dataset_name = None,
    data_dir = "./data",
    train_file = "/content/data/textaug_train_rsicd.jsonl",
    validation_file = "/content/data/textaug_valid_rsicd.jsonl",
    #train_file = "/content/data/textaug_train_nostro.jsonl",
    #validation_file = "/content/data/textaug_valid_nostro.jsonl",
    max_train_samples = None,
    max_eval_samples = None,
    overwrite_cache = False,
    validation_split_percentage = 5,
    preprocessing_num_workers = 2,
    augment_images = True,
    augment_captions = True,
    captions_per_image = 5,
)

augmentation_args = ImageAugmentationArguments()

Some utility functions for training:

In [ ]:
class TrainState(train_state.TrainState):
    dropout_rng: jnp.ndarray

    def replicate(self):
        return jax_utils.replicate(self).replace(dropout_rng=shard_prng_key(self.dropout_rng))


def write_train_metric(summary_writer, train_metrics, train_time, step):
    summary_writer.scalar("train_time", train_time, step)

    train_metrics = get_metrics(train_metrics)
    for key, vals in train_metrics.items():
        tag = f"train_{key}"
        for i, val in enumerate(vals):
            summary_writer.scalar(tag, val, step - len(vals) + i + 1)


def write_eval_metric(summary_writer, eval_metrics, step):
    for metric_name, value in eval_metrics.items():
        summary_writer.scalar(f"eval_{metric_name}", value, step)


def create_learning_rate_fn(
    train_ds_size: int, train_batch_size: int, num_train_epochs: int, warmup_ratio: float, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    steps_per_epoch = train_ds_size // train_batch_size
    num_train_steps = steps_per_epoch * num_train_epochs
    num_warmup_steps = int(num_train_steps * warmup_ratio)
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn

# utils
def mb_item(x):
    return x.item() if hasattr(x, "item") else x

def make_batch(samples):
    batch = {k:jnp.array(v) for k,v in samples.items()}
    return batch


## Training

In [ ]:
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.DEBUG,
)

In [ ]:
# Load pretrained model and tokenizer

# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, cache_dir=model_args.cache_dir)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
else:
    config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

processor = CLIPProcessor.from_pretrained(model_args.model_name_or_path)
tokenizer = processor.tokenizer
if model_args.model_name_or_path:
    model = FlaxCLIPModel.from_pretrained(
        model_args.model_name_or_path, config=config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )
else:
    model = FlaxCLIPModel.from_config(
        config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
[codecarbon INFO @ 17:49:15] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 17:49:15] RAM : 4.75 W during 15.01 s [measurement time: 0.0031]
[codecarbon INFO @ 17:49:15] Energy consumed for all GPUs : 0.000132 kWh. Total GPU Power : 31.690055814921042 W
[codecarbon DEBUG @ 17:49:15] GPU : 31.69 W during 15.01 s [measurement time: 0.0044]
[codecarbon INFO @ 17:49:15] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5

In [ ]:
config = model.config
# Initialize torchvision transforms and jit them for faster processing
preprocess = Transform(config.vision_config.image_size, data_args.augment_images, augmentation_args)
preprocess = torch.jit.script(preprocess)

eval_preprocess = Transform(config.vision_config.image_size, False, augmentation_args)
eval_preprocess = torch.jit.script(eval_preprocess)

# Initialize the image-text dataset
train_dataset = ImageTextDataset(
    data_args.data_dir,
    "train",
    captions_per_image=data_args.captions_per_image,
    augment_captions=data_args.augment_captions,
    transform=preprocess,
)

eval_dataset = ImageTextDataset(
    data_args.data_dir,
    "valid",
    captions_per_image=1,
    transform=eval_preprocess,
)

# Enable tensorboard only on the master node
has_tensorboard = is_tensorboard_available()
if has_tensorboard and jax.process_index() == 0:
    try:
        from flax.metrics.tensorboard import SummaryWriter

        summary_writer = SummaryWriter(log_dir=Path(training_args.output_dir))
    except ImportError as ie:
        has_tensorboard = False
        logger.warning(
            f"Unable to display metrics through TensorBoard because some package are not installed: {ie}"
        )
else:
    logger.warning(
        "Unable to display metrics through TensorBoard because the package is not installed: "
        "Please run pip install tensorboard to enable."
    )

# Use collate function to tokenizer the text and convert the processed images to numpy
def collate_fn(examples):
    pixel_values = torch.stack([example[0] for example in examples]).numpy()
    captions = [example[1] for example in examples]
    inputs = tokenizer(captions, max_length=32, padding="max_length", return_tensors="np", truncation=True)

    batch = {
        "pixel_values": pixel_values,
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
    }

    return batch

# Store some constant
num_epochs = int(training_args.num_train_epochs)
train_batch_size = int(training_args.per_device_train_batch_size) * jax.device_count() * training_args.gradient_accumulation_steps
eval_batch_size = int(training_args.per_device_eval_batch_size) * jax.device_count()
steps_per_epoch = len(train_dataset) // train_batch_size
total_train_steps = steps_per_epoch * num_epochs

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=data_args.preprocessing_num_workers,
    persistent_workers=True,
    drop_last=True,
    collate_fn=collate_fn,
)

eval_loader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
    num_workers=data_args.preprocessing_num_workers,
    persistent_workers=True,
    drop_last=True,
    collate_fn=collate_fn,
)


1 input files for train split found
1 input files for valid split found


In [ ]:
'''
# enable wandb tracking
has_wandb = find_spec("wandb") is not None
if jax.process_index() == 0 and has_wandb and ("wandb" in training_args.report_to):
    try:
        import wandb
        if training_args.run_name is None:
            run_name = training_args.output_dir.split("/")[-1]
        else:
            run_name = training_args.run_name
        wandb.init(
            name=run_name,
            entity="wandb",
            project="hf-flax-clip-rsicd",
            sync_tensorboard=True
        )
        wandb.config.update(training_args)
        wandb.config.update(model_args)
        wandb.config.update(data_args)
    except ImportError as e:
        print(e)
        has_wandb = False
'''
has_wandb = False


# Initialize our training
rng = jax.random.PRNGKey(training_args.seed)
rng, dropout_rng = jax.random.split(rng)

# Create learning rate schedule
linear_decay_lr_schedule_fn = create_learning_rate_fn(
    len(train_dataset),
    train_batch_size,
    training_args.num_train_epochs,
    training_args.warmup_ratio,
    training_args.learning_rate,
)

# We use Optax's "masking" functionality to not apply weight decay
# to bias and LayerNorm scale parameters. decay_mask_fn returns a
# mask boolean with the same structure as the parameters.
# The mask is True for parameters that should be decayed.
# Note that this mask is specifically adapted for FlaxGPT2.
# For other models, one should correct the layer norm parameter naming
# accordingly.
def decay_mask_fn(params):
    flat_params = traverse_util.flatten_dict(params)
    flat_mask = {
        path: (path[-1] != "bias" and path[-2:] not in [("ln_1", "scale"), ("ln_2", "scale"), ("ln_f", "scale")])
        for path in flat_params
    }
    return traverse_util.unflatten_dict(flat_mask)

# create optimizer
if training_args.adafactor:
    # We use the default parameters here to initialize adafactor,
    # For more details about the parameters please check https://github.com/deepmind/optax/blob/ed02befef9bf81cbbf236be3d2b0e032e9ed4a40/optax/_src/alias.py#L74
    optimizer = optax.adafactor(
        learning_rate=linear_decay_lr_schedule_fn,
    )
else:
    optimizer = optax.adamw(
        learning_rate=linear_decay_lr_schedule_fn,
        b1=training_args.adam_beta1,
        b2=training_args.adam_beta2,
        eps=training_args.adam_epsilon,
        weight_decay=training_args.weight_decay,
        mask=decay_mask_fn,
    )
    optimizer = optax.chain(
        optax.clip_by_global_norm(training_args.max_grad_norm),
        optimizer
    )
if training_args.gradient_accumulation_steps > 1:
    optimizer = optax.MultiSteps(optimizer, training_args.gradient_accumulation_steps)
grad_accum_steps = training_args.gradient_accumulation_steps

# Setup train state
state = TrainState.create(apply_fn=model.__call__, params=model.params, tx=optimizer, dropout_rng=dropout_rng)

if training_args.resume_from_checkpoint:
    state = restore_checkpoint(training_args.resume_from_checkpoint, state)
    resume_step = mb_item(state.step)
else:
    resume_step = 0

In [ ]:
def cross_entropy(logits, axis):
    logprobs = jax.nn.log_softmax(logits, axis=axis)
    nll = jnp.diag(logprobs)
    ce = -jnp.mean(nll)
    return ce

def clip_loss(similarity):
    loss = (cross_entropy(similarity, axis=0) + cross_entropy(similarity, axis=1)) / 2
    return loss

# Define gradient update step fn
def train_step(state, batch):
    dropout_rng, new_dropout_rng = jax.random.split(state.dropout_rng)

    def compute_loss(params):
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = clip_loss(logits)
        return loss

    grad_fn = jax.value_and_grad(compute_loss)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")

    new_state = state.apply_gradients(grads=grad, dropout_rng=new_dropout_rng)

    metrics = {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step // grad_accum_steps)}
    metrics = jax.lax.pmean(metrics, axis_name="batch")

    return new_state, metrics

# Define eval fn
def eval_step(params, batch):
    logits = model(**batch, params=params, train=False)[0]
    loss = clip_loss(logits)

    # summarize metrics
    metrics = {"loss": loss}
    metrics = jax.lax.pmean(metrics, axis_name="batch")
    return metrics

# Create parallel version of the train and eval step
p_train_step = jax.pmap(train_step, "batch", donate_argnums=(0,))
p_eval_step = jax.pmap(eval_step, "batch")

# Replicate the train state on each device
state = state.replicate()

In [ ]:
logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_epochs}")
logger.info(f"  Instantaneous batch size per device = {training_args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed and grad_accum) = {train_batch_size}")
logger.info(f"  Total optimization steps = {total_train_steps}")



if not training_args.skip_memory_metrics:
    server = jax.profiler.start_server(9999)

train_time = 0
train_metrics = []
epochs = tqdm(range(num_epochs), desc=f"Epoch ... (1/{num_epochs})", position=0)
for epoch in epochs:
    # ======================== Training ================================
    train_start = time.time()

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    steps_per_epoch = len(train_dataset) // train_batch_size
    # train
    steps_trained_progress_bar = tqdm(range(steps_per_epoch), desc="Training...", position=1,
                                        leave=False, initial=(resume_step // grad_accum_steps))
    for step, batch in enumerate(train_loader):
        cur_step = epoch * (len(train_dataset) // train_batch_size) + step
        # skip to the step from which we are resuming
        if cur_step < resume_step:
            continue

        batch = shard(make_batch(batch))

        state, train_metric = p_train_step(state, batch)
        train_metrics.append(train_metric)
        if step % grad_accum_steps == 0:
            steps_trained_progress_bar.update(1)

        if cur_step % (training_args.logging_steps * grad_accum_steps)== 0 and cur_step > 0:
            # Save metrics
            train_metric = unreplicate(train_metric)
            train_time += time.time() - train_start
            if has_tensorboard and jax.process_index() == 0:
                write_train_metric(summary_writer, train_metrics, train_time, cur_step)
            if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
                # TODO: add accumulation of metrics
                _metrics = {k if k=="learning_rate" else f"train_{k}":mb_item(v.mean()) for k, v in train_metric.items()}
                wandb.log({"training_step":cur_step, **_metrics}, commit=True)

            epochs.write(
                f"Step... ({cur_step} | Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()})"
            )

            train_metrics = []

    if True:
        # ======================== Evaluating ==============================
        eval_metrics = []
        eval_steps = len(eval_dataset) // eval_batch_size
        for batch in tqdm(eval_loader, desc="Evaluating...", position=2, leave=False):
            # Model forward
            batch = shard(make_batch(batch))
            metrics = p_eval_step(state.params, batch)
            eval_metrics.append(metrics)

        # normalize eval metrics
        eval_metrics = get_metrics(eval_metrics)
        eval_metrics = jax.tree.map(jnp.mean, eval_metrics)

        # Print metrics and update progress bar
        desc = f"Step... ({cur_step} | Eval Loss: {eval_metrics['loss']})"
        epochs.write(desc)
        epochs.desc = desc

        # Save metrics
        if has_tensorboard and jax.process_index() == 0:
            # cur_step = epoch * (len(train_dataset) // train_batch_size)
            write_eval_metric(summary_writer, eval_metrics, cur_step)
        if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
            _metrics = {f"eval_{k}":mb_item(v) for k, v in eval_metrics.items()}
            wandb.log({"eval_step":cur_step, **_metrics})

    # save checkpoint after each epoch
    if jax.process_index() == 0:
        save_dir = f"{training_args.output_dir}/ckpt-{epoch}"
        model.save_pretrained(
            save_dir,
            params=unreplicate(state.params),
            push_to_hub=False, # training_args.push_to_hub, # we don't push intermediate steps
            commit_message=f"Saving weights and logs at epoch {epoch}",
            repo_name_or_path=training_args.output_dir
        )
        if model_args.save_optimizer:
            save_checkpoint(training_args.output_dir, unreplicate(state), cur_step, keep=training_args.save_total_limit, overwrite=True)
        if training_args.save_total_limit is not None:
            rotate_checkpoints(training_args.output_dir, training_args.save_total_limit)

tracker.stop()

INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 30360
INFO:__main__:  Num Epochs = 5
INFO:__main__:  Instantaneous batch size per device = 32
INFO:__main__:  Total train batch size (w. parallel, distributed and grad_accum) = 32
INFO:__main__:  Total optimization steps = 4740


Epoch ... (1/5):   0%|          | 0/5 [00:00<?, ?it/s]

Training...:   0%|          | 0/948 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[codecarbon INFO @ 17:49:31] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 17:49:31] RAM : 4.75 W during 15.07 s [measurement time: 0.0567]
[codecarbon INFO @ 17:49:31] Energy consumed for all GPUs : 0.000266 kWh. Total GPU Power : 31.5810077293174 W
[codecarbon DEBUG @ 17:49:31] GPU : 31.58 W during 15.25 s [measurement time: 0.1552]
[codecarbon INFO @ 17:49:31] Energy consumed for all CPUs : 0.000360 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 17:49:31] CPU : 42.50 W during 15.49 s [measurement time: 0.1312]
[codecarbon INFO @ 17:49:31] 0.000666 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 17:49:31] last_duration=15.065488576889038
------------------------
[codecarbon INFO @ 17:49:46] 

Step... (50 | Loss: 2.1641931533813477, Learning Rate: 2.63713104686758e-08)


...

[codecarbon INFO @ 18:20:47] Energy consumed for RAM : 0.002511 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:20:47] RAM : 4.75 W during 15.00 s [measurement time: 0.0074]
[codecarbon INFO @ 18:20:47] Energy consumed for all GPUs : 0.034804 kWh. Total GPU Power : 68.97653484316366 W
[codecarbon DEBUG @ 18:20:47] GPU : 68.98 W during 15.01 s [measurement time: 0.0092]
[codecarbon INFO @ 18:20:47] Energy consumed for all CPUs : 0.022485 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:20:47] CPU : 42.50 W during 15.03 s [measurement time: 0.0053]
[codecarbon INFO @ 18:20:47] 0.059800 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:20:47] last_duration=14.999187707901001
------------------------


Evaluating...:   0%|          | 0/23 [00:00<?, ?it/s]

Step... (4739 | Eval Loss: 3.3341622352600098)


[codecarbon INFO @ 18:21:02] Energy consumed for RAM : 0.002531 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:21:02] RAM : 4.75 W during 14.96 s [measurement time: 0.0027]
[codecarbon INFO @ 18:21:02] Energy consumed for all GPUs : 0.035043 kWh. Total GPU Power : 57.57194393101421 W
[codecarbon DEBUG @ 18:21:02] GPU : 57.57 W during 14.97 s [measurement time: 0.0089]
[codecarbon INFO @ 18:21:02] Energy consumed for all CPUs : 0.022662 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:21:02] CPU : 42.50 W during 14.98 s [measurement time: 0.0019]
[codecarbon INFO @ 18:21:02] 0.060236 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:21:02] last_duration=14.958220720291138
------------------------
[codecarbon INFO @ 18:21:03] Energy consumed for RAM : 0.002533 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:21:03] RAM : 4.75 W during 1.69 s [measurement time: 0.0027]
[codecarbon INFO @ 18:21:03] Energy consumed for all GPUs : 0.035058 kWh. T

0.008362919960150563

In [ ]:
# Write dictionary to JSON file
with open((training_args.output_dir + "/train_args.json"), "w") as json_file:
    json.dump(training_args.to_dict(), json_file, indent=4)

print(f"TrainingArguments saved")

!mv /content/tmp/ /content/gdrive/MyDrive/


TrainingArguments saved


In [ ]:
tracker.stop()

[codecarbon WARNING @ 18:21:43] Tracker already stopped !
[codecarbon INFO @ 18:21:43] Energy consumed for RAM : 0.002585 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:21:43] RAM : 4.75 W during 39.29 s [measurement time: 0.0014]
[codecarbon INFO @ 18:21:43] Energy consumed for all GPUs : 0.035400 kWh. Total GPU Power : 31.2919223560984 W
[codecarbon DEBUG @ 18:21:43] GPU : 31.29 W during 39.31 s [measurement time: 0.0176]
[codecarbon INFO @ 18:21:43] Energy consumed for all CPUs : 0.023146 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:21:43] CPU : 42.50 W during 39.33 s [measurement time: 0.0113]
[codecarbon INFO @ 18:21:43] 0.061131 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:21:43] last_duration=39.291815757751465
------------------------
[codecarbon DEBUG @ 18:21:43] EmissionsData(timestamp='2024-06-18T18:21:43', project_name='RS-ZSL-UNIVPM', run_id='0cbbd381-5d97-455f-b56f-bc04f7f3a255', duration=1962.1978840827942, emissions=0.00848193

0.008481938676461104